In [16]:
import re
from collections import Counter, defaultdict, deque
from copy import deepcopy
from dataclasses import dataclass, field
from functools import cache
from heapq import heappop, heappush
from itertools import combinations, permutations, product
from math import atan2, ceil, inf, lcm, pi
from typing import *

import black
import jupyter_black
import networkx as nx
from icecream import ic
from more_itertools import chunked
from parse import parse
from primefac import primefac

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def first(iterable):
    return next(iter(iterable))

In [30]:
# Day 1: The Tyranny of the Rocket Equation
def fuel_needed(mass):
    fuel = mass // 3 - 2
    if fuel <= 0:
        return 0
    return fuel_needed(fuel) + fuel


modules = [int(x) for x in open("2019/1.txt").read().splitlines()]
print(f"Part 1: {sum(module // 3 - 2 for module in modules)}")
print(f"Part 2: {sum(fuel_needed(module) for module in modules)}")

Part 1: 3295424
Part 2: 4940279


In [95]:
# Day 2: 1202 Program Alarm
def address(pointer):
    return memory[pointer]


def reg(pos):
    return memory[memory[pos]]


def add(ip):
    memory[address(ip + 3)] = reg(ip + 1) + reg(ip + 2)


def mul(ip):
    memory[address(ip + 3)] = reg(ip + 1) * reg(ip + 2)


def run_program():
    ip = 0
    instruction = memory[ip]
    while instruction != 99:
        instructions[instruction][0](ip)
        ip += instructions[instruction][1]
        instruction = memory[ip]


def part1():
    memory[1] = 12
    memory[2] = 2
    run_program()


def part2():
    global memory
    memory = program.copy()
    for noun, verb in product(range(100), range(100)):
        memory[1] = noun
        memory[2] = verb
        run_program()
        if address(0) == 19690720:
            break
        memory = program.copy()
    return noun, verb


# opcode: (instruction, increase of ip)
instructions = {1: (add, 4), 2: (mul, 4)}

program = [int(x) for x in open("2019/2.txt").read().split(",")]
memory = program.copy()
part1()
print(f"Part 1: {address(0)}")  # 5098658
noun, verb = part2()
print(f"Part 2: {100*noun + verb}") # 5064

Part 1: 5098658
Part 2: 5064


In [205]:
# Day 3: Crossed Wires
def create_wire(moves):
    x, y = 0, 0
    cur_step = 1
    wire = {}
    dx = dict(zip("LRUD", [-1, 1, 0, 0]))
    dy = dict(zip("LRUD", [0, 0, 1, -1]))
    for move in moves:
        dir, steps = move
        for step in range(steps):
            x += dx[dir]
            y += dy[dir]
            wire[(x, y)] = cur_step + step
        cur_step += steps
    return wire


def parse_line(line):
    return [tuple((move[0], int(move[1:]))) for move in line.split(",")]


def origo_distance(a):
    return sum(abs(x) for x in a)


a, b = open("2019/3.txt").read().splitlines()
moves_a = parse_line(a)
moves_b = parse_line(b)
# moves_a = parse_line("R75,D30,R83,U83,L12,D49,R71,U7,L72")
# moves_b = parse_line("U62,R66,U55,R34,D71,R55,D58,R83")
a = create_wire(moves_a)
b = create_wire(moves_b)
intersections = {point: a[point] + b[point] for point in a if point in b}
print(f"Part 1: {origo_distance(min(intersections, key=origo_distance))}")  # 2180
print(f"Part 2: {min(intersections.values())}")  # 112316

Part 1: 2180
Part 2: 112316


In [282]:
# Day 4: Secure Container
def no_decrease(number):
    number = str(number)
    return all(b >= a for a, b in zip(number, number[1:]))


def two_consecutive(number):
    number = str(number)
    return any(b == a for a, b in zip(number, number[1:]))


def strip_repeats(number):
    """Returns a string where all numbers that are repeated three times or more are
    removed. E.g. '122233' -> '133'."""
    result = str(number)
    for x in re.findall(r"(.)\1{2,}", str(number)):
        result = result.replace(x, "")
    return result


matches = 0
for number in range(172851, 675869 + 1):
    if no_decrease(number) and two_consecutive(number):
        matches += 1

print(f"Part 1: {matches}")  # 1660

matches = 0
for number in range(172851, 675869 + 1):
    if no_decrease(number) and two_consecutive(number):
        no_long_repeats = strip_repeats(number)
        if no_long_repeats and two_consecutive(int(no_long_repeats)):
            matches += 1
print(f"Part 2: {matches}")  # 1135

Part 1: 1688
Part 2: 1135


In [6]:
# Day 5: Sunny with a change of Asteroids
def parameter(offset, modes):
    return memory[memory[ip + offset]] if modes[-offset] == "0" else memory[ip + offset]


def destination(offset, modes):
    return memory[ip + offset] if modes[-offset] == "0" else ip + offset


def add(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) + parameter(2, modes)
    ip += 4


def mul(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) * parameter(2, modes)
    ip += 4


def poke(modes):
    global ip
    memory[destination(1, modes)] = input_value
    ip += 2


def peek(modes):
    global ip
    mem_address = destination(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def less_than(modes):
    global ip
    if parameter(1, modes) < parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if parameter(1, modes) == parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def run_program(program, input):
    global ip, input_value, memory
    instructions = {
        1: add,
        2: mul,
        3: poke,
        4: peek,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
    }
    input_value = input
    memory = program.copy()
    outputs, ip = [], 0
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        result = instructions[opcode](modes)  # Execute instruction
        if result:
            outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Last two digits
    return outputs


program = [int(x) for x in open("2019/5.txt").read().split(",")]
print("Part 1:", first(run_program(program, 1)))  # 13294380
print("Part 2:", first(run_program(program, 5)))  # 11460760

Part 1: 13294380
Part 2: 11460760


In [27]:
# Day 6: Universal Orbit Map
lines = open("2019/6.txt").read().splitlines()

G = nx.Graph()
for line in lines:
    u, v = line.split(")")
    G.add_edge(u, v)
orbits = nx.single_target_shortest_path_length(G, "COM")
print(f"Part 1: {sum(dist for _, dist in orbits)}")  # 135690
print(f"Part 2: {nx.shortest_path_length(G, 'YOU', 'SAN') - 2}")  # 298

Part 1: 135690
Part 2: 298


In [4]:
# Day 7: Amplification Circuit
@dataclass
class Amplifier:
    inputs: List[int]
    outputs: List[int]
    memory: List[int]
    ip: int = 0
    running: bool = True


def parameter(offset, modes):
    return memory[memory[ip + offset]] if modes[-offset] == "0" else memory[ip + offset]


def destination(offset, modes):
    return memory[ip + offset] if modes[-offset] == "0" else ip + offset


def add(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) + parameter(2, modes)
    ip += 4


def mul(modes):
    global ip
    memory[destination(3, modes)] = parameter(1, modes) * parameter(2, modes)
    ip += 4


def read_input(modes):
    global ip, inputs
    if inputs:
        memory[destination(1, modes)] = inputs.pop(0)
    else:
        raise RuntimeError("No values in inputs")
    ip += 2


def output(modes):
    global ip
    mem_address = destination(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not parameter(1, modes):
        ip = parameter(2, modes)
    else:
        ip += 3


def less_than(modes):
    global ip
    if parameter(1, modes) < parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if parameter(1, modes) == parameter(2, modes):
        memory[destination(3, modes)] = 1
    else:
        memory[destination(3, modes)] = 0
    ip += 4


def run_program(amplifier):
    global ip, memory, inputs
    instructions = {
        1: add,
        2: mul,
        3: read_input,
        4: output,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
    }
    memory = amplifier.memory.copy()
    inputs = amplifier.inputs.copy()
    ip = amplifier.ip
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        if instructions[opcode] == read_input and not inputs:
            # Waiting for inputs. Store program state and let the next amplifier run
            amplifier.memory = memory.copy()
            amplifier.inputs = inputs.copy()
            amplifier.ip = ip
            return
        result = instructions[opcode](modes)  # Execute instruction
        if instructions[opcode] == output:
            amplifier.outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Next opcode
    amplifier.running = False


def run_amplifiers(phase_settings, program, one_round=True):
    amplifiers = [Amplifier([setting], [], program) for setting in phase_settings]
    prev_output = 0  # Input to first amplifier
    while any(amp.running for amp in amplifiers):
        for amplifier in amplifiers:
            amplifier.inputs.append(prev_output)
            run_program(amplifier)
            prev_output = amplifier.outputs.pop(0)
        if one_round:
            return prev_output
    return prev_output


program = [int(x) for x in open("2019/7.txt").read().split(",")]
part1 = max(
    run_amplifiers(phase_settings, program) for phase_settings in permutations(range(5))
)
print(f"Part 1: {part1}")  # 95757
part2 = max(
    run_amplifiers(phase_settings, program, one_round=False)
    for phase_settings in permutations(range(5,10))
)
print(f"Part 2: {part2}")  # 4275738

Part 1: 95757
Part 2: 4275738


In [106]:
# Day 8: Space Image Format
def fewest_zeros(layers):
    "Return index for layer with fewest 0:s"
    min_zeros = 10**6
    index = -1
    for i, layer in enumerate(layers):
        if layer.count(0) < min_zeros:
            min_zeros = layer.count(0)
            index = i
    return index


def pixel_value(layers, position):
    TRANSPARENT = 2
    for layer in range(len(layers)):
        if layers[layer][position] == TRANSPARENT:
            continue
        else:
            return layers[layer][position]
    return TRANSPARENT


def print_image(image):
    for row in range(heigth):
        for col in range(width):
            print("⬛️⬜️"[image[row * width + col] * 2], end="")  # CFLUL
        print()


digits = open("2019/8.txt").read().strip()
width, heigth = 25, 6
digits = [int(x) for x in digits]
pixels_per_layer = width * heigth
num_layers = len(digits) // (pixels_per_layer)

layers = [
    digits[layer * pixels_per_layer : (layer + 1) * pixels_per_layer]
    for layer in range(num_layers)
]

index = fewest_zeros(layers)
part1 = layers[index].count(1) * layers[index].count(2)
print(f"Part 1: {part1}")

image = []
for position in range(pixels_per_layer):
    image.append(pixel_value(layers, position))

print("Part 2:")
print_image(image)

Part 1: 1935
Part 2:
⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬛⬛⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬛⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛


In [4]:
# Day 9: Sensor Boost
@dataclass
class VirtualMachine:
    inputs: List[int]
    outputs: List[int]
    memory: DefaultDict[int, int]
    ip: int = 0
    relative_base: int = 0
    running: bool = True


def address(offset, modes):
    POSITION = "0"
    IMMEDIATE = "1"
    RELATIVE = "2"
    if modes[-offset] == POSITION:
        return memory[ip + offset]
    elif modes[-offset] == IMMEDIATE:
        return ip + offset
    elif modes[-offset] == RELATIVE:
        return memory[ip + offset] + relative_base
    else:
        raise ValueError("Invalid mode", modes[-offset], offset, modes)


def add(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] + memory[address(2, modes)]
    ip += 4


def mul(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] * memory[address(2, modes)]
    ip += 4


def read_input(modes):
    global ip, inputs
    if inputs:
        memory[address(1, modes)] = inputs.pop(0)
    else:
        raise RuntimeError("No values in inputs")
    ip += 2


def output(modes):
    global ip
    mem_address = address(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def less_than(modes):
    global ip
    if memory[address(1, modes)] < memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if memory[address(1, modes)] == memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def relative_base_offset(modes):
    global ip, relative_base
    relative_base += memory[address(1, modes)]
    ip += 2


def run_program(vm):
    global ip, memory, inputs, relative_base
    instructions = {
        1: add,
        2: mul,
        3: read_input,
        4: output,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
        9: relative_base_offset,
    }
    memory = vm.memory.copy()
    inputs = vm.inputs.copy()
    ip = vm.ip
    relative_base = vm.relative_base
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        if instructions[opcode] == read_input and not inputs:
            # Waiting for inputs. Store program state and let the next amplifier run
            vm.memory = memory.copy()
            vm.inputs = inputs.copy()
            vm.ip = ip
            vm.relative_base = relative_base
            return
        result = instructions[opcode](modes)  # Execute instruction
        if instructions[opcode] == output:
            vm.outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Next opcode
    vm.running = False


def run_virtual_machine(input_value=0):
    vm = VirtualMachine([input_value], [], program)
    run_program(vm)
    return vm


def create_program(text):
    program = defaultdict(int)
    for pos, x in enumerate(text.split(",")):
        program[pos] = int(x)
    return program


program = create_program(open("2019/9.txt").read())
vm = run_virtual_machine(input_value=1)
print(f"Part 1: {first(vm.outputs)}")  # 2453265701
vm = run_virtual_machine(input_value=2)
print(f"Part 2: {first(vm.outputs)}")  # 80805

Part 1: 2453265701
Part 2: 80805


In [156]:
# Day 10: Monitoring Station
from math import sqrt


@dataclass(frozen=True)
class Point:
    x: int
    y: int


def on_line(start, end, point):
    """True if point is exactly on the line between start and end.

    Implementation from https://stackoverflow.com/questions/328107/how-can-you-determine-a-point-is-between-two-other-points-on-a-line-segment"""
    crossproduct = (point.y - start.y) * (end.x - start.x) - (point.x - start.x) * (
        end.y - start.y
    )
    if abs(crossproduct) > 0.000001:
        return False

    dotproduct = (point.x - start.x) * (end.x - start.x) + (point.y - start.y) * (
        end.y - start.y
    )
    if dotproduct < 0:
        return False

    squared_length = (end.x - start.x) ** 2 + (end.y - start.y) ** 2
    if dotproduct > squared_length:
        return False
    return True


def direct_line_of_sight(a, b, points):
    points = points - {a, b}
    return not any(on_line(a, b, point) for point in points)


def can_see_count(asteroids):
    result = Counter()
    for a, b in combinations(asteroids, r=2):
        if direct_line_of_sight(a, b, asteroids):
            result.update({a, b})
    return result


def angle(start, point):
    "Angle between start and point. Min value straight up."
    x = point.x - start.x
    y = point.y - start.y
    result = atan2(y, x)
    if -pi < result < -pi / 2:
        # Top left quadrant
        return result + 2 * pi
    else:
        return result


def can_see(point, asteroids):
    return [a for a in asteroids if direct_line_of_sight(point, a, asteroids)]


lines = open("2019/10.txt").read().splitlines()
asteroids = {
    Point(x, y)
    for y, line in enumerate(lines)
    for x, pos in enumerate(line)
    if pos == "#"
}

c = can_see_count(asteroids)
print(f"Part 1: {max(c.values())}")  # 292 (10s execution time)

# Part 2
start = c.most_common(1)[0][0]
seen = can_see(start, asteroids)
seen.sort(key=lambda x: angle(start, x))
target = seen[199]
print(f"Part 2: {target.x * 100 + target.y}")  # 317

Part 1: 292
Part 2: 317


In [4]:
# Day 11: Space Police
@dataclass
class VirtualMachine:
    inputs: List[int]
    outputs: List[int]
    memory: DefaultDict[int, int]
    ip: int = 0
    relative_base: int = 0
    running: bool = True


def address(offset, modes):
    POSITION = "0"
    IMMEDIATE = "1"
    RELATIVE = "2"
    if modes[-offset] == POSITION:
        return memory[ip + offset]
    elif modes[-offset] == IMMEDIATE:
        return ip + offset
    elif modes[-offset] == RELATIVE:
        return memory[ip + offset] + relative_base
    else:
        raise ValueError("Invalid mode", modes[-offset], offset, modes)


def add(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] + memory[address(2, modes)]
    ip += 4


def mul(modes):
    global ip
    memory[address(3, modes)] = memory[address(1, modes)] * memory[address(2, modes)]
    ip += 4


def read_input(modes):
    global ip, inputs
    if inputs:
        memory[address(1, modes)] = inputs.pop(0)
    else:
        raise RuntimeError("No values in inputs")
    ip += 2


def output(modes):
    global ip
    mem_address = address(1, modes)
    ip += 2
    return memory[mem_address]


def jump_if_true(modes):
    global ip
    if memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def jump_if_false(modes):
    global ip
    if not memory[address(1, modes)]:
        ip = memory[address(2, modes)]
    else:
        ip += 3


def less_than(modes):
    global ip
    if memory[address(1, modes)] < memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def equals(modes):
    global ip
    if memory[address(1, modes)] == memory[address(2, modes)]:
        memory[address(3, modes)] = 1
    else:
        memory[address(3, modes)] = 0
    ip += 4


def relative_base_offset(modes):
    global ip, relative_base
    relative_base += memory[address(1, modes)]
    ip += 2


def run_program(vm):
    global ip, memory, inputs, relative_base
    instructions = {
        1: add,
        2: mul,
        3: read_input,
        4: output,
        5: jump_if_true,
        6: jump_if_false,
        7: less_than,
        8: equals,
        9: relative_base_offset,
    }
    memory = vm.memory.copy()
    inputs = vm.inputs.copy()
    ip = vm.ip
    relative_base = vm.relative_base
    opcode = int(str(memory[ip])[-2:])  # Last two digits of instruction field
    while opcode != 99:
        modes = "000" + str(memory[ip])[:-2]  # Everything but last two digits, padded
        if instructions[opcode] == read_input and not inputs:
            # Waiting for inputs. Store program state and let the next amplifier run
            vm.memory = memory.copy()
            vm.inputs = inputs.copy()
            vm.ip = ip
            vm.relative_base = relative_base
            return
        result = instructions[opcode](modes)  # Execute instruction
        if instructions[opcode] == output:
            vm.outputs.append(result)
        opcode = int(str(memory[ip])[-2:])  # Next opcode
    vm.running = False


def create_program(text):
    program = defaultdict(int)
    for pos, x in enumerate(text.split(",")):
        program[pos] = int(x)
    return program


def draw(program, start_color):
    panels = defaultdict(int)
    dirs = [
        (-1, 0),
        (0, 1),
        (1, 0),
        (0, -1),
    ]  # dx, dy for up, right, down, left
    x = y = dir_index = 0
    vm = VirtualMachine([start_color], [], program)
    while vm.running:
        run_program(vm)
        # Paint (first output)
        panels[(x, y)] = vm.outputs.pop(0)
        # Turn (second output)
        turn_right = vm.outputs.pop(0)
        if turn_right == 1:
            dir_index = (dir_index + 1) % 4
        else:
            dir_index = (dir_index - 1) % 4
        # Move
        dx, dy = dirs[dir_index]
        x += dx
        y += dy
        vm.inputs = [panels[(x, y)]]
    return panels


program = create_program(open("2019/11.txt").read())
BLACK, WHITE = 0, 1

print(f"Part 1: {len(draw(program, BLACK))}")  # 1876
print("Part 2:")
panels = draw(program, WHITE)
xmax, ymax = max(panels)
for x in range(xmax + 1):
    for y in range(ymax + 1):
        print("⬛" if panels[(x, y)] else "⬜️", end="")
    print()

Part 1: 1876
Part 2:
⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬛⬛⬛⬜️⬜️⬜️⬜️⬛⬛⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬛⬜️⬛⬛⬛⬜️⬜️⬜️⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬛⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬛⬜️⬛⬜️⬜️⬜️⬜️
⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬛⬜️⬛⬜️⬜️⬜️⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬜️⬜️⬜️⬛⬛⬛⬜️⬜️⬛⬛⬜️⬜️⬛⬛⬛⬛⬜️


In [2]:
# Day 12: The N-Body Problem
@dataclass
class Moon:
    position: Tuple[int, int, int]
    velocity: List[int] = field(default_factory=lambda: [0, 0, 0])


def apply_gravity(a, b):
    def adjust_velocity(axis):
        if a.position[axis] < b.position[axis]:
            a.velocity[axis] += 1
            b.velocity[axis] -= 1
        elif a.position[axis] > b.position[axis]:
            a.velocity[axis] -= 1
            b.velocity[axis] += 1

    for axis in range(3):
        adjust_velocity(axis)


def apply_velocity(moon):
    new_position = tuple(moon.position[i] + moon.velocity[i] for i in range(3))
    moon.position = new_position


def time_step(moons):
    for a, b in combinations(moons, r=2):
        apply_gravity(a, b)
    for moon in moons:
        apply_velocity(moon)


def potential_energy(moon):
    return sum(abs(coord) for coord in moon.position)


def kinetec_energy(moon):
    return sum(abs(coord) for coord in moon.velocity)


def total_energy(moon):
    return potential_energy(moon) * kinetec_energy(moon)


lines = open("2019/12.txt").read().splitlines()
moons = [Moon(tuple(int(x) for x in re.findall(r"-?\d+", line))) for line in lines]
for _ in range(1000):
    time_step(moons)
print(f"Part 1: {sum(total_energy(moon) for moon in moons)}")  # 7758

# Part 2
moons = [Moon(tuple(int(x) for x in re.findall(r"-?\d+", line))) for line in lines]
originals = deepcopy(moons)
steps_until_axis_repeats = [0, 0, 0]
for step in range(1, 1_000_000):
    time_step(moons)
    for axis in range(3):
        if not steps_until_axis_repeats[axis] and all(
            moon.position[axis] == original.position[axis]
            and moon.velocity[axis] == original.velocity[axis]
            for moon, original in zip(moons, originals)
        ):
            steps_until_axis_repeats[axis] = step
    if all(steps_until_axis_repeats):
        break
lcm(*steps_until_axis_repeats)

# Multiply all items in list
print(f"Part 2: {lcm(*steps_until_axis_repeats)}")  # 354540398381256

Part 1: 7758
Part 2: 354540398381256


In [4]:
# Day 13: Care Package - Run Day 11 first to enable IntCode computer
def create_tiles(vm):
    return {(x, y): id for x, y, id in chunked(vm.outputs, 3)}


def blocks(tiles):
    return list(tiles.values()).count(2)


def display(tiles):
    xmax = max(tiles.keys(), key=lambda pt: pt[0])[0]
    ymax = max(tiles.keys(), key=lambda pt: pt[1])[1]
    print(tiles[-1, 0])  # Score
    for y in range(ymax + 1):
        for x in range(xmax + 1):
            print(".#&_o"[tiles[(x, y)]], end="")
        print()


def play_game(program):
    "Play game until completion. Return score."

    def y_position(tiles, tile_id):
        return first(x for (x, y), id in tiles.items() if id == tile_id)

    def play_round(vm, joystick):
        vm.inputs = [joystick]
        run_program(vm)

    PADDLE, BALL = 3, 4
    LEFT, NEUTRAL, RIGHT = -1, 0, 1
    vm = VirtualMachine([], [], program)
    vm.memory[0] = 2
    joystick = 0
    play_round(vm, joystick)
    tiles = create_tiles(vm)
    while vm.running and blocks(tiles):
        play_round(vm, joystick)
        tiles = create_tiles(vm)
        ball = y_position(tiles, BALL)
        paddle = y_position(tiles, PADDLE)
        if ball < paddle:
            joystick = LEFT
        elif ball > paddle:
            joystick = RIGHT
        else:
            joystick = NEUTRAL
    return tiles[-1, 0]


program = create_program(open("2019/13.txt").read())
vm = VirtualMachine([], [], program)
run_program(vm)
print(f"Part 1: {blocks(create_tiles(vm))}")  # 200

program = create_program(open("2019/13.txt").read())
print(f"Part 2: {play_game(program)}")  # 9803 (20s running time)

Part 1: 200
Part 2: 9803


In [434]:
# Day 14: Space Stoichiometry
def parse_lines(lines):
    def parse_quantity(quantity):
        amount, chemical = quantity.split(" ")
        return (chemical, int(amount))

    costs = {}
    for line in lines:
        inputs, output = line.split(" => ")
        inputs = inputs.split(", ")
        chemical, batch_size = parse_quantity(output)
        costs[chemical] = [batch_size] + [parse_quantity(x) for x in inputs]
    return costs


def chemicals_needed(fuel_to_produce=1):
    def produced_by_ore(chemical):
        return costs[chemical][1][0] == "ORE"

    needs = defaultdict(int)
    for (chemical, amount) in costs["FUEL"][1:]:
        needs[chemical] = amount * fuel_to_produce

    for _ in range(100):
        previous = needs.copy()
        for chemical, amount in previous.items():
            if produced_by_ore(chemical):
                continue
            batch_size = costs[chemical][0]
            batches = ceil(amount / batch_size)
            for produced_chem, produced_cost in costs[chemical][1:]:
                needs[produced_chem] += batches * produced_cost
            needs[chemical] -= batches * batch_size
        if needs == previous:
            return needs
    raise ValueError("Could not unfold needs after 100 iterations")


def ore_consumed(needs):
    ore = 0
    for chemical, amount_needed in needs.items():
        batch_size = costs[chemical][0]
        batch_cost = costs[chemical][1][1]
        batches = ceil(amount_needed / batch_size)
        ore += batches * batch_cost
    return ore


def fuel_produced(ore_available):
    lower_bound = ore_available // ore_consumed(chemicals_needed(1))
    for upper_bound in range(lower_bound, lower_bound * 10, 1000):
        if ore_consumed(chemicals_needed(upper_bound)) > ore_available:
            lower_bound = upper_bound - 1000
            break

    for fuel in range(upper_bound, lower_bound, -1):
        if ore_consumed(chemicals_needed(fuel)) < ore_available:
            return fuel


lines = open("2019/14.txt").read().splitlines()
costs = parse_lines(lines)
print(f"Part 1: {ore_consumed(chemicals_needed(1))}")  # 741927

fuel = fuel_produced(ore_available=1_000_000_000_000)
print(f"Part 2: {fuel}")  # 2371699

Part 1: 741927
Part 2: 2371699


In [5]:
# Day 15: Oxygen System - Run Day 11 first to enable IntCode computer
def create_grid(program):
    "Create a grid by walking around hugging the left wall until we return to the starting point."
    vm = VirtualMachine([], [], program)
    grid = defaultdict(lambda: 3)
    row, column = 0, 0
    dirs = [N, W, S, E]
    current_direction = 0
    for _ in range(100_000):
        # Try to take a step forward
        vm.inputs = [dirs[current_direction]]
        dr, dc = directions[dirs[current_direction]]
        run_program(vm)
        result = vm.outputs.pop()
        grid[(row + dr, column + dc)] = result
        if result == WALL:
            current_direction = (current_direction - 1) % 4  # Turn right
        else:
            # We could move forward
            row += dr
            column += dc
            current_direction = (current_direction + 1) % 4  # Turn left
        if (row, column) == (0, 0):
            break
    return grid


def display(grid):
    row_min = min(grid.keys(), key=lambda point: point[0])[0]
    col_min = min(grid.keys(), key=lambda point: point[1])[1]
    row_max = max(grid.keys(), key=lambda point: point[0])[0]
    col_max = max(grid.keys(), key=lambda point: point[1])[1]
    symbols = "#.O-"
    for row in range(row_min, row_max + 1):
        for col in range(col_min, col_max + 1):
            print(symbols[grid[(row, col)]], end="")
        print()


def neighbors(position, grid):
    row, column = position
    candidates = [
        (row - 1, column),
        (row + 1, column),
        (row, column - 1),
        (row, column + 1),
    ]
    return tuple(pos for pos in candidates if grid[pos] in (PATH, OXYGEN))


def minutes_to_fill(grid):
    "Time to fill entire grid with oxygen."

    def fill(grid):
        "Fill one minute of oxygen"
        oxygen_coords = [key for key, value in grid.items() if value == OXYGEN]
        for coord in oxygen_coords:
            for neighbor in neighbors(coord, grid):
                grid[neighbor] = OXYGEN
        return grid

    for minutes in range(1000):
        grid = fill(grid)
        if PATH not in grid.values():
            return minutes + 1


def a_star(start, goal, grid):
    def heuristic(current, goal) -> float:
        (x1, y1) = current
        (x2, y2) = goal
        return abs(x1 - x2) + abs(y1 - y2)

    frontier = []
    heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = defaultdict(lambda: inf)
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        if heuristic(current, goal) == 0:
            break
        for next in neighbors(current, grid):
            new_cost = cost_so_far[current] + 1
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                heappush(frontier, (priority, next))
                came_from[next] = current

    return cost_so_far[goal]


N, S, W, E = 1, 2, 3, 4
directions = {N: (-1, 0), S: (1, 0), W: (0, -1), E: (0, 1)}
WALL, PATH, OXYGEN = 0, 1, 2
grid = create_grid(create_program(open("2019/15.txt").read()))
start = (0, 0)
goal = first(key for key, value in grid.items() if value == OXYGEN)

print(f"Part 1: {a_star(start, goal, grid)}")  # 366
print(f"Part 2: {minutes_to_fill(grid)}")  # 384

Part 1: 366
Part 2: 384


In [381]:
# Day 16: Flawed Frequency Transmission
def pattern(position):
    "Return pattern according Day 16, 0-based position."

    def complete_pattern(position):
        base = (0, 1, 0, -1)
        pos = 0
        while True:
            for _ in range(position):
                yield base[pos]
            pos = (pos + 1) % len(base)

    iter = complete_pattern(position + 1)
    next(iter)
    yield from iter


def fft_phase(signal):
    result = ""
    for pos, digit in enumerate(signal):
        digit_result = 0
        for digit, multiplier in zip(signal, pattern(pos)):
            digit_result += int(digit) * multiplier
        result += str(digit_result)[-1]
    return result


def part2(signal, repetitions=10_000, rounds=100):
    signal = open("2019/16.txt").read().strip()
    signal = signal * repetitions
    start_position = int(signal[:7])
    signal = [int(x) for x in signal]
    table = [signal[start_position:]]
    # Digits in the second half of the number (i.e. 5678 in the number 12345678) can be
    # calculated with: value(pos, round) = value(pos, round - 1) + value(pos + 1, round)
    for round in range(1, rounds + 1):
        table.append(table[round - 1])
        # Last digit never changes, start one off the end and move backwards
        for position in range(len(table[round]) - 2, -1, -1):
            table[round][position] = (
                table[round - 1][position] + table[round][position + 1]
            ) % 10
    # 41402171, 41 s
    return "".join(str(x) for x in table[100][:8])


signal = open("2019/16.txt").read().strip()
for _ in range(100):
    signal = fft_phase(signal)
print(f"Part 1: {signal[:8]}")  # 22122816, 12s

signal = open("2019/16.txt").read().strip()
print(f"Part 2: {part2(signal)}")  # 41402171, 50s

Part 1: 22122816
Part 2: 41402171


In [472]:
# Day 17: Set and Forget - Run day 11 to enable IntCode
def create_grid(camera_output):
    grid = {}
    for row, line in enumerate(camera_output):
        for col, char in enumerate(line):
            grid[(row, col)] = char
    return grid


def alignment_parameters():
    def intersection(row, col):
        to_check = [
            (row, col),
            (row, col - 1),
            (row, col + 1),
            (row - 1, col),
            (row + 1, col),
        ]
        return all(grid[(r, c)] == "#" for (r, c) in to_check)

    return sum(
        row * col
        for row, col in product(range(1, max(grid)[0]), range(1, max(grid)[1]))
        if intersection(row, col)
    )


def input_codes():
    def encode(line):
        return [ord(x) for x in line] + [10]

    # Manually followed the grid in 17-map.txt
    ROUTINE = "A,A,B,C,C,A,C,B,C,B"
    A = "L,4,L,4,L,6,R,10,L,6"
    B = "L,12,L,6,R,10,L,6"
    C = "R,8,R,10,L,6"
    FEED = "n"

    return encode(ROUTINE) + encode(A) + encode(B) + encode(C) + encode(FEED)


# Part 1
program = create_program(open("2019/17.txt").read())
vm = VirtualMachine(inputs=[], outputs=[], memory=program)
run_program(vm)
camera_output = "".join([chr(x) for x in vm.outputs]).strip().splitlines()
grid = create_grid(camera_output)
print(f"Part 1: {alignment_parameters()}")  # 3448

# Part 2
program[0] = 2
vm = VirtualMachine(inputs=input_codes(), outputs=[], memory=program)
run_program(vm)
print(f"Part 2: {vm.outputs[-1]}")  # 762405

Part 1: 3448
Part 2: 762405


In [65]:
# Day 18: Many-Worlds Interpretation
def parse_grid(lines):
    grid = {}
    for row, line in enumerate(lines):
        for column, char in enumerate(line.strip()):
            grid[(row, column)] = char
    return grid


def define_paths(nodes):
    def path_length_and_doors(start, goal):
        "Length between two keys and doors found on the way."

        def bfs(start, goal):
            def moves(position):
                (r, c) = position
                candidates = [
                    (r + 1, c),
                    (r - 1, c),
                    (r, c + 1),
                    (r, c - 1),
                ]
                return [coord for coord in candidates if grid[coord] != "#"]

            frontier = [start]
            came_from = {start: None}
            while frontier:
                current = frontier.pop(0)
                if current == goal:
                    break
                for move in moves(current):
                    if move not in came_from:
                        frontier.append(move)
                        came_from[move] = current
            return came_from

        start = first(coord for coord, value in grid.items() if value == start)
        goal = first(coord for coord, value in grid.items() if value == goal)
        came_from = bfs(start, goal)
        doors = set()
        if goal not in came_from.keys():
            return 0, doors
        current = goal
        steps = 0
        while current != start:
            steps += 1
            if grid[current].isupper():
                doors.add(grid[current])
            current = came_from[current]
        return steps, doors

    paths = {}
    for start, end in permutations(nodes, 2):
        length, doors = path_length_and_doors(start, end)
        if length:
            paths[(start, end)] = (length, doors)
    return paths


def heuristic(current, goal):
    "Minimum estimation of cost to get from a to goal. Also used to determine when we have reached goal (heuristic returns 0)."
    _, keys = current
    _, wanted_keys = goal
    return len(wanted_keys) - len(keys)


def cost(current, next):
    start, _ = current
    end, _ = next
    return paths[(start, end)][0]


def moves(state):
    current, keys = state
    destinations = set(wanted_keys) - keys
    for destination in destinations:
        steps, doors = paths[(current, destination)]
        if all(door.lower() in keys for door in doors):
            yield (destination, keys | {destination})


def a_star(start, goal):
    frontier = []
    heappush(frontier, (0, start))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        if heuristic(current, goal) == 0:
            break
        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost  # + heuristic(next, goal)
                heappush(frontier, (priority, next))

    return cost_so_far[current]


lines = open("2019/18.txt").read().splitlines()
grid = parse_grid(lines)
wanted_keys = {value for value in grid.values() if value.islower()}
paths = define_paths(wanted_keys | {"@"})
start = ("@", frozenset())
goal = (_, wanted_keys)
print(f"Part 1: {a_star(start, goal)}")  # 7430, 5s

# Day 18, part 2 - new state and definitions for cost and moves, same a_star as in part 1
def cost(current, next):
    current_bots, _ = current
    next_bots, _ = next
    for current, next in zip(current_bots, next_bots):
        if current != next:
            return paths[(current, next)][0]


def moves(current):
    def individual_moves(bot, keys):
        destinations = wanted_keys - keys
        for destination in destinations:
            if (bot, destination) not in paths.keys():
                continue
            steps, doors = paths[(bot, destination)]
            if all(door.lower() in keys for door in doors):
                yield (destination, keys | {destination})

    bots, keys = current
    bots = list(bots)
    # All moves for one bot, then move on to the next only changing one bot at a time
    for index, bot in enumerate(bots):
        for new_bot, new_keys in individual_moves(bot, keys):
            bots[index] = new_bot
            yield tuple(bots), new_keys
            bots[index] = bot


lines = open("2019/18-2.txt").read().splitlines()
grid = parse_grid(lines)
wanted_keys = {value for value in grid.values() if value.islower()}
start_positions = ("0", "1", "2", "3")
paths = define_paths(wanted_keys | set(start_positions))
start = (start_positions, frozenset())
goal = (_, wanted_keys)
print(f"Part 2: {a_star(start, goal)}")  # 1864, 5s

Part 1: 7430
Part 2: 1864


In [6]:
# Day 19: Tractor Beam
# Part 1
program = create_program(open("2019/19.txt").read())
grid = {}
for x, y in product(range(50), range(50)):
    vm = VirtualMachine([x, y], [], program)
    run_program(vm)
    grid[(x, y)] = vm.outputs[0]
# display(grid)
print(f"Part 1: {sum(grid.values())}")

# Part 2
def display(grid):
    row_min = min(grid.keys(), key=lambda point: point[0])[0]
    col_min = min(grid.keys(), key=lambda point: point[1])[1]
    row_max = max(grid.keys(), key=lambda point: point[0])[0]
    col_max = max(grid.keys(), key=lambda point: point[1])[1]
    symbols = ".#"
    for row in range(row_min, row_max + 1):
        for col in range(col_min, col_max + 1):
            print(symbols[grid[(row, col)]], end="")
        print()


# will show what we are dealing with
# for x, y in product(range(100), range(100)):
#     vm = VirtualMachine([x, y], [], program)
#     run_program(vm)
#     grid[(x, y)] = vm.outputs[0]
# display(grid)

# We have two lines starting at 0. y = kx. What are two k values?
# line 1: x = 70, y = 79
# line 2: x = 70, y = 97

# Get better estimates for k values
def calculate_y(x, k, prev=0, numbers_to_try=10):
    "k needs to be lower than actual value"
    y_est = int(x * k)
    for y in range(y_est, y_est + numbers_to_try):
        vm = VirtualMachine([x, y], [], program)
        run_program(vm)
        if y == y_est:
            assert vm.outputs[0] == prev, f"{prev}"
        if vm.outputs[0] != prev:
            return y
    return 0


# First iteration
k1 = 79 / 70 * 0.95
k2 = 97 / 70
x = 100
k1 = calculate_y(x, k1, prev=0) / x
k2 = calculate_y(x, k2, prev=1) / x
# Second iteration
k1 = k1 * 0.99
k2 = k2 * 0.99
x = 100_000
k1 = calculate_y(x, k1, prev=0, numbers_to_try=2000) / x
k2 = calculate_y(x, k2, prev=1, numbers_to_try=1000) / x
# Third iteration
k1 = k1 * 0.999
k2 = k2 * 0.999
x = 1_000_000
k1 = calculate_y(x, k1, prev=0, numbers_to_try=2000) / x
k2 = calculate_y(x, k2, prev=1, numbers_to_try=2000) / x
# We now have good k-values 1.118792, 1.391519
# Use these k-values for a faster method to tell i x,y is affected
def affected(x, y):
    k1, k2 = 1.118792, 1.391519
    return int(k1 * x < y < k2 * x)


# Solving arithmetically gives:
x2 = (100 * (k2 + 1)) / (k2 - k1)
y1 = k1 * x2
x1 = x2 - 100
y21 = k2 * x1
y22 = y1 + 100
# x1, x2, y1, y21, y22 = 776.9 876.9 981.1 1081.1 1081.1

# But 7760969 is too high and 6965773 (previous incorrect calculation) too low

# We have something that is close, but not quite correct. Let's just explore all points
# that are close to (776, 980) and find the first one that fulfills the criteria
def points():
    for x in range(765, 777):
        for y in range(970, 985):
            if (
                sum(
                    affected(x + x_delta, y + y_delta)
                    for x_delta, y_delta in product(range(100), range(100))
                )
                == 100 * 100
            ):
                yield x, y


x, y = first(points())
print(f"Part 2: {x * 10_000 + y}")  # 7720975 correct answer

# From https://www.reddit.com/r/adventofcode/comments/ecogl3/comment/fbdmn5n/
# Beautiful solution
# x = y = 0
# while not affected(x + 99, y):
#     y += 1
#     while not affected(x, y + 99):
#         x += 1
# x * 10_000 + y

Part 1: 215
Part 2: 7720975


In [70]:
# Day 20: Donut Maze
def candidates(row, col):
    return [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]


def find_other_letter(lines, row, col):
    assert lines[row][col].isupper()
    return first(
        lines[r][c] for (r, c) in candidates(row, col) if lines[r][c].isupper()
    )


def neighbors(grid, row, col):
    result = set()
    for (r, c) in candidates(row, col):
        if (r, c) in grid.keys():
            if grid[(r, c)] == ".":
                result.add((r, c))
            else:
                assert grid[(r, c)].isupper()
                result |= letter_coordinates(grid, grid[(r, c)])
    return result


def letter_coordinates(grid, letter_combination):
    result = set()
    coordinates = {
        (r, c) for (r, c), string in grid.items() if string == letter_combination
    }
    for (row, col) in coordinates:
        for (r, c) in candidates(row, col):
            if (r, c) in grid.keys():
                if grid[(r, c)] == ".":
                    result.add((r, c))
    return result


def create_grid(lines):
    raw_grid = {}
    for row, line in enumerate(lines):
        for col, char in enumerate(line):
            if char == ".":
                raw_grid[(row, col)] = "."
            elif char.isupper():
                label = "".join(sorted(char + find_other_letter(lines, row, col)))
                raw_grid[(row, col)] = label
                if label == "AA" or label == "ZZ":
                    print(row, col)
    grid = {}
    for (r, c), value in raw_grid.items():
        if value == ".":
            grid[(r, c)] = neighbors(raw_grid, r, c)
    return grid


def dijkstra(start, goal):
    frontier = []
    heappush(frontier, (0, start))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        if current == goal:
            break
        for next in grid[current]:
            new_cost = cost_so_far[current] + 1
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                heappush(frontier, (new_cost, next))

    return cost_so_far[current]


lines = open("2019/20.txt").read().splitlines()
grid = create_grid(lines)
start, goal = (2, 69), (61, 2)
dijkstra(start, goal)

0 69
1 69
61 0
61 1


522

'.'